In [1]:
configs = {
  "fs.adl.oauth2.access.token.provider.type": "ClientCredential",
  "fs.adl.oauth2.client.id": "4ab956bb-5a0b-45b3-a87b-22424b89ea36",
  "fs.adl.oauth2.credential": ".2SD44SJ2nti=QtOym[@anQ[cXiW?6v.",
  "fs.adl.oauth2.refresh.url": "https://login.microsoftonline.com/5d7e4366-1b9b-45cf-8e79-b14b27df46e1/oauth2/token"}

dbutils.fs.mount(
  source = "adl://imagerepo.azuredatalakestore.net/dogImages",
  mount_point  = "/mnt/dogImages",
  extra_configs = configs)

dbutils.fs.mount(
  source = "adl://imagerepo.azuredatalakestore.net/cifar-10-batches-py",
  mount_point  = "/mnt/cifar-10-batches-py",
  extra_configs = configs)

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob2 import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 134)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('/dbfs/mnt/dogImages/dogImages/train/')
valid_files, valid_targets = load_dataset('/dbfs/mnt/dogImages/dogImages/valid/')
test_files, test_targets = load_dataset('/dbfs/mnt/dogImages/dogImages/test/')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("/dbfs/mnt/dogImages/dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
There are 134 total dog categories.
There are 8405 total dog images.

There are 6717 training dog images.
There are 845 validation dog images.
There are 843 test dog images.

In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files)
valid_tensors = paths_to_tensor(valid_files)
test_tensors = paths_to_tensor(test_files)

0%| | 0/6717 [00:00<?, ?it/s] 0%| | 4/6717 [00:00<02:51, 39.06it/s] 0%| | 8/6717 [00:00<02:52, 38.89it/s] 0%| | 13/6717 [00:00<02:50, 39.42it/s] 0%| | 17/6717 [00:00<02:53, 38.52it/s] 0%| | 21/6717 [00:00<03:03, 36.53it/s] 0%| | 25/6717 [00:00<02:58, 37.43it/s] 0%| | 29/6717 [00:00<02:56, 37.96it/s] 0%| | 33/6717 [00:00<03:20, 33.26it/s] 1%| | 37/6717 [00:01<03:28, 32.07it/s] 1%| | 41/6717 [00:01<03:37, 30.69it/s] 1%| | 45/6717 [00:01<03:38, 30.51it/s] 1%| | 49/6717 [00:01<03:59, 27.83it/s] 1%| | 53/6717 [00:01<03:48, 29.18it/s] 1%| | 57/6717 [00:01<03:38, 30.52it/s] 1%| | 61/6717 [00:01<03:58, 27.94it/s] 1%| | 65/6717 [00:02<03:52, 28.59it/s] 1%| | 69/6717 [00:02<04:51, 22.80it/s] 1%| | 72/6717 [00:02<04:47, 23.10it/s] 1%| | 75/6717 [00:02<04:35, 24.12it/s] 1%| | 78/6717 [00:02<04:21, 25.43it/s] 1%| | 82/6717 [00:02<04:06, 26.92it/s] 1%|▏ | 86/6717 [00:02<03:58, 27.84it/s] 1%|▏ | 89/6717 [00:03<04:21, 25.39it/s] 1%|▏ | 93/6717 [00:03<04:11, 26.31it/s] 1%|▏ | 97/6717 [00:03<04:02, 27.26it/s] 1%|▏ | 100/6717 [00:03<03:57, 27.85it/s] 2%|▏ | 103/6717 [00:03<03:59, 27.64it/s] 2%|▏ | 106/6717 [00:03<04:23, 25.11it/s] 2%|▏ | 109/6717 [00:03<04:17, 25.71it/s] 2%|▏ | 112/6717 [00:03<04:12, 26.19it/s] 2%|▏ | 115/6717 [00:04<05:06, 21.51it/s] 2%|▏ | 118/6717 [00:04<04:43, 23.27it/s] 2%|▏ | 122/6717 [00:04<04:22, 25.11it/s] 2%|▏ | 125/6717 [00:04<06:32, 16.77it/s] 2%|▏ | 128/6717 [00:04<06:01, 18.22it/s] 2%|▏ | 132/6717 [00:04<05:14, 20.93it/s] 2%|▏ | 135/6717 [00:05<04:48, 22.84it/s] 2%|▏ | 138/6717 [00:05<04:31, 24.20it/s] 2%|▏ | 141/6717 [00:05<05:43, 19.15it/s] 2%|▏ | 145/6717 [00:05<05:02, 21.75it/s] 2%|▏ | 148/6717 [00:05<04:41, 23.34it/s] 2%|▏ | 151/6717 [00:05<04:31, 24.16it/s] 2%|▏ | 154/6717 [00:05<04:21, 25.08it/s] 2%|▏ | 157/6717 [00:05<04:13, 25.84it/s] 2%|▏ | 161/6717 [00:06<04:04, 26.80it/s] 2%|▏ | 165/6717 [00:06<03:59, 27.39it/s] 3%|▎ | 168/6717 [00:06<04:35, 23.78it/s] 3%|▎ | 171/6717 [00:06<06:45, 16.15it/s] 3%|▎ | 174/6717 [00:06<06:10, 17.64it/s] 3%|▎ | 177/6717 [00:06<05:31, 19.75it/s] 3%|▎ | 181/6717 [00:07<04:55, 22.10it/s] 3%|▎ | 184/6717 [00:07<04:38, 23.48it/s] 3%|▎ | 187/6717 [00:07<04:30, 24.13it/s] 3%|▎ | 190/6717 [00:07<04:56, 21.98it/s] 3%|▎ | 193/6717 [00:07<07:27, 14.59it/s] 3%|▎ | 197/6717 [00:07<06:05, 17.84it/s] 3%|▎ | 201/6717 [00:08<05:20, 20.36it/s] 3%|▎ | 204/6717 [00:08<04:51, 22.31it/s] 3%|▎ | 207/6717 [00:08<04:33, 23.79it/s] 3%|▎ | 210/6717 [00:08<05:03, 21.43it/s] 3%|▎ | 213/6717 [00:08<06:13, 17.43it/s] 3%|▎ | 216/6717 [00:08<05:41, 19.05it/s] 3%|▎ | 219/6717 [00:08<05:29, 19.74it/s] 3%|▎ | 222/6717 [00:09<05:00, 21.62it/s] 3%|▎ | 225/6717 [00:09<04:49, 22.45it/s] 3%|▎ | 228/6717 [00:09<04:32, 23.83it/s] 3%|▎ | 231/6717 [00:09<04:19, 25.02it/s] 3%|▎ | 234/6717 [00:09<07:34, 14.26it/s] 4%|▎ | 237/6717 [00:09<06:34, 16.42it/s] 4%|▎ | 240/6717 [00:10<05:52, 18.38it/s] 4%|▎ | 244/6717 [00:10<05:03, 21.35it/s] 4%|▎ | 247/6717 [00:10<04:41, 23.02it/s] 4%|▎ | 250/6717 [00:10<04:46, 22.56it/s] 4%|▍ | 253/6717 [00:10<04:42, 22.86it/s] 4%|▍ | 256/6717 [00:10<04:22, 24.57it/s] 4%|▍ | 260/6717 [00:10<04:02, 26.60it/s] 4%|▍ | 263/6717 [00:10<04:20, 24.77it/s] 4%|▍ | 266/6717 [00:10<04:11, 25.66it/s] 4%|▍ | 270/6717 [00:11<03:45, 28.64it/s] 4%|▍ | 274/6717 [00:11<03:56, 27.22it/s] 4%|▍ | 277/6717 [00:11<04:22, 24.58it/s] 4%|▍ | 280/6717 [00:11<09:20, 11.49it/s] 4%|▍ | 283/6717 [00:12<07:44, 13.86it/s] 4%|▍ | 286/6717 [00:12<08:11, 13.08it/s] 4%|▍ | 289/6717 [00:12<06:48, 15.73it/s] 4%|▍ | 292/6717 [00:12<05:55, 18.07it/s] 4%|▍ | 296/6717 [00:12<05:11, 20.61it/s] 4%|▍ | 300/6717 [00:12<04:43, 22.62it/s] 5%|▍ | 303/6717 [00:13<06:43, 15.91it/s] 5%|▍ | 306/6717 [00:13<06:48, 15.67it/s] 5%|▍ | 309/6717 [00:13<05:59, 17.81it/s] 5%|▍ | 312/6717 [00:13<05:21, 19.95it/s] 5%|▍ | 315/6717 [00:13<04:52, 21.86it/s] 5%|▍ | 318/6717 [00:14<08:00, 13.33it/s] 5%|▍ | 320/6717 [00:14<08:49, 12.08it/s] 5%|▍ | 323/6717 [00:14<07:3

In [5]:
def extract_VGG16(tensor):
	from keras.applications.vgg16 import VGG16, preprocess_input
	return VGG16(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_VGG19(tensor):
	from keras.applications.vgg19 import VGG19, preprocess_input
	return VGG19(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_Resnet50(tensor):
	from keras.applications.resnet50 import ResNet50, preprocess_input
	return ResNet50(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_Xception(tensor):
	from keras.applications.xception import Xception, preprocess_input
	return Xception(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_InceptionV3(tensor):
	from keras.applications.inception_v3 import InceptionV3, preprocess_input
	return InceptionV3(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

In [6]:
bottleneck_features_train = extract_Resnet50(train_tensors)

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

/databricks/python/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
 warnings.warn('The output shape of `ResNet50(include_top=False)` '

In [7]:
bottleneck_features_valid = extract_Resnet50(valid_tensors)

In [8]:
bottleneck_features_test = extract_Resnet50(test_tensors)

In [9]:
np.save("/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataTrain", bottleneck_features_train)
np.save("/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataValid", bottleneck_features_valid)
np.save("/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataTest", bottleneck_features_test)

In [10]:
np.save("/dbfs/mnt/dogImages/bottleneck_features/TrainTargets", train_targets)
np.save("/dbfs/mnt/dogImages/bottleneck_features/ValidTargets", valid_targets)
np.save("/dbfs/mnt/dogImages/bottleneck_features/TestTargets", test_targets)

In [11]:
np.save("/dbfs/mnt/dogImages/bottleneck_features/DogNames", dog_names)